In [ ]:
import sys
sys.path.append('../')

In [ ]:
from tank import Tank, Hole, create_holes
import ipywidgets as widgets
from ipycanvas import canvas
from IPython.display import display, Markdown, Latex
import time
from demo import *

In [3]:
class TankDemo:
    def __init__(self, canvas, water_q, n_holes, d_holes):
        self.canvas: canvas.Canvas = canvas
        self.tank = Tank(create_holes(n_holes, d_holes), water_q)
        self.depth = self.tank.get_depth()
        self.update_count = 1
        self.init_ui()

    def init_ui(self):
        self.nSlider = widgets.IntSlider(
            value=25,
            min=5,
            max=50,
            description="Nr of Holes",
            continuous_update=False
        )
        self.nText = widgets.Label(value="[ ]")
        self.nBox = widgets.HBox([self.nSlider, self.nText])
        self.dSlider = widgets.FloatSlider(
            value=2,
            min=0.1,
            max=5,
            description="Diameter",
            continuous_update=False
        )
        self.dText = widgets.Label(value="[cm]")
        self.dBox = widgets.HBox([self.dSlider, self.dText])
        self.nSlider.observe(self.update_holes)
        self.dSlider.observe(self.update_holes)

        self.qSlider = widgets.FloatSlider(
            value=0.06,
            step=0.01,
            min=0.01,
            max=1,
            description="Water flow Q",
            continuous_update=False
        )
        self.qText = widgets.HTMLMath(value="$[m^3s^{-1}]$")
        self.qBox = widgets.HBox([self.qSlider, self.qText])
        self.maxDepthSlider = widgets.FloatSlider(
            value=1,
            step=0.1,
            min=0.5,
            max=5,
            description="Tank Depth",
            continuous_update=False
        )
        self.mdText = widgets.Label(value="[m]")
        self.mdBox = widgets.HBox([self.maxDepthSlider, self.mdText])
        self.qSlider.observe(self.update_tank)
        self.maxDepthSlider.observe(self.update_tank)

        self.tankTitle = widgets.HTML("<h1>Tank</h1>")
        self.holeTitle = widgets.HTML("<h1>Holes</h1>")
        self.tankDetails = widgets.VBox([self.tankTitle, self.qBox, self.mdBox])
        self.holeDetails = widgets.VBox([self.holeTitle, self.nBox, self.dBox])
        display(widgets.HBox([self.tankDetails, self.holeDetails]))
        self.output = widgets.Output()
        display(self.canvas)
        display(self.output)
        self.canvas.on_client_ready(self.draw)
        self.update_output()

    def update_holes(self, args):
        n_prev = len(self.tank.holes)
        n_now = self.nSlider.value

        d = self.dSlider.value

        if n_prev != n_now:
            diff = n_now - n_prev
            if diff > 0:
                for i in range(diff):
                    self.tank.add_hole(Hole(d))
            else:
                for i in range(abs(diff)):
                    self.tank.pop_hole()
        if d != self.tank.holes[0].d:
            self.tank.set_diameter(d)
        self.update_output()

    def update_tank(self, args):
        self.tank.q = self.qSlider.value
        self.tank.depth = self.maxDepthSlider.value
        self.update_output()

    def update_output(self):
        self.output.clear_output()
        self.depth = self.tank.get_depth()
        with self.output:
            display(widgets.HTML(f'<b>Depth h</b> = {self.depth:.2f}m'))
        self.update_count += 1
        self.draw()

    def draw(self):
        self.update_count = 1
        self.canvas.clear()
        rect = self.tank.get_dimensions(50, 50)
        x_0 = rect[0]
        y_0 = rect[1]
        x_1 = x_0 + rect[2]
        y_1 = y_0 + rect[3]
        
        self.canvas.height = y_1 + 50

        if self.depth > self.tank.depth:
            wy_0 = y_0 - 5
            self.canvas.stroke_style = 'red'
            self.canvas.stroke_text("OVERFLOW!", x_1 + 15, y_0 - 5)
            self.canvas.stroke_style = 'black'
        else:
            wy_0 = y_1 - self.depth * 100
        gradient = self.canvas.create_linear_gradient(
            x_0, wy_0, x_1, y_1,
            # List of color stops
            [
                (0, '#CAFAD4'),
                (1, '#90D3D6'),
            ],
        )
        self.canvas.fill_style = gradient
        self.canvas.global_alpha = 0.75
        self.canvas.fill_rect(x_0 + 1, wy_0, rect[2] - 1.5, y_1 - wy_0 - 1)
        
        """if 15 <= len(self.tank.holes) < 40:
            holes = self.tank.draw_holes(5, x_0, y_1, 20)
        elif len(self.tank.holes) >= 40:
            holes = self.tank.draw_holes(7, x_0, y_1, 20)
        elif len(self.tank.holes) < 15:
            holes = self.tank.draw_holes(1, x_0, y_1, 20)
        """
        self.canvas.global_alpha = 1
        
        line = [x_0 - 10, wy_0, x_0 - 10, y_1]
        self.canvas.stroke_line(*line)
        self.canvas.stroke_text("h", x_0 - 30, (wy_0 + y_1) // 2)
        
        holes = self.tank.alt_draw_holes(15, x_0, y_1, 20)
        for hole in holes:
            self.canvas.fill_rect(*hole)
        
        self.canvas.line_width = 3.0
        self.canvas.begin_path()
        self.canvas.move_to(x_0, y_0)
        self.canvas.line_to(x_0, y_1)
        self.canvas.line_to(x_1, y_1)
        self.canvas.line_to(x_1, y_0)
        self.canvas.stroke()
        self.canvas.line_width = 1.0